<a href="https://colab.research.google.com/github/ArunRK7Codie/Sentiment-Analysis/blob/master/sentiment_analysis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
! wget https://www.kaggle.com/kazanova/sentiment140/download

--2020-05-08 09:14:21--  https://www.kaggle.com/kazanova/sentiment140/download
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?returnUrl=%2Fkazanova%2Fsentiment140%2Fdata [following]
--2020-05-08 09:14:21--  https://www.kaggle.com/account/login?returnUrl=%2Fkazanova%2Fsentiment140%2Fdata
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download’

download                [ <=>                ]   8.75K  --.-KB/s    in 0.03s   

2020-05-08 09:14:22 (278 KB/s) - ‘download’ saved [8956]

